<div style="font-family:'JetBrains Mono', monospace; text-align:center; color:#da251d;">
    
#  510 Spadina Streetcar Improvement Model
<div style="text-align:center;"><strong>by:</strong> Sandesh Bhandari | I made this as part of my GGR424 Final Paper</div>
<div style="text-align:center;"><strong>Date:</strong> November 29, 2025</div>
</div>

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set global style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['figure.dpi'] = 300

# Color palette (Red for current/old, Green for proposed/new)
CURRENT_RED = '#e74c3c'
PROPOSED_GREEN = '#2ecc71'
PALE_GREEN = '#c5f29e'
DEEP_GREEN = '#073435'
COOL_GREY = '#95a5a6'
DARK_SLATE = '#2c3e50'


In [18]:
#LOADING AND PREPARING DATA
# !!!I RENAMED THE DELAY DATA CSV FILE TO delay.csv FOR EASIER HANDLING!!!
def load_delay_data(delay_path='delay.csv'):
    """Load and filter delay data for 510 Spadina"""
    df = pd.read_csv(delay_path)
    
    # Filter for 510 Spadina -
    df['Line'] = df['Line'].astype(str).str.strip()
    spadina = df[df['Line'].str.contains('510', case=False, na=False)]
    
    # Convert time and date
    spadina['DateTime'] = pd.to_datetime(spadina['Date'] + ' ' + spadina['Time'], errors='coerce')
    spadina['Hour'] = spadina['DateTime'].dt.hour
    spadina['MinDelay'] = pd.to_numeric(spadina['Min Delay'], errors='coerce')
    spadina['MinGap'] = pd.to_numeric(spadina['Min Gap'], errors='coerce')    
    return spadina

def load_gtfs_data(gtfs_folder='gtfs/'):
    """Load GTFS data for 510 Spadina"""
    routes = pd.read_csv(f'{gtfs_folder}routes.txt')
    trips = pd.read_csv(f'{gtfs_folder}trips.txt')
    stop_times = pd.read_csv(f'{gtfs_folder}stop_times.txt')
    stops = pd.read_csv(f'{gtfs_folder}stops.txt')
    
    # Find 510 route - 
    routes['route_short_name'] = routes['route_short_name'].astype(str).str.strip()
    route_510 = routes[routes['route_short_name'].str.contains('510', case=False, na=False)]
    
    if len(route_510) == 0:
        # Try route_id
        routes['route_id'] = routes['route_id'].astype(str).str.strip()
        route_510 = routes[routes['route_id'].str.contains('510', case=False, na=False)]
    
    route_id = route_510['route_id'].iloc[0]
    
    # Get trips for this route
    trips_510 = trips[trips['route_id'] == route_id]
    
    # Get stop times for these trips
    stop_times_510 = stop_times[stop_times['trip_id'].isin(trips_510['trip_id'])]
    
    # Merge with stops to get locations
    stop_times_510 = stop_times_510.merge(stops, on='stop_id')
    
    return stop_times_510, stops, route_id


In [19]:

#BASELINE DATA BASED ON GTFS AND DELAY DATA


def calculate_baseline_travel_times(stop_times_510):
    """Calculate current travel times from GTFS scheduled times"""
    travel_times = []
    
    # Handle times like "25:30:00" (past midnight)
    def time_to_minutes(t):
        try:
            if isinstance(t, str):
                parts = t.split(':')
                return int(parts[0]) * 60 + int(parts[1]) + int(parts[2])/60
            else:
                return None
        except:
            return None
    
    print(f"Processing {len(stop_times_510['trip_id'].unique())} unique trips...")
    
    for trip_id in stop_times_510['trip_id'].unique():
        trip_data = stop_times_510[stop_times_510['trip_id'] == trip_id].sort_values('stop_sequence')
        
        if len(trip_data) > 5:  # Need at least a few stops
            # Get first and last stop times
            start_time = time_to_minutes(trip_data.iloc[0]['arrival_time'])
            end_time = time_to_minutes(trip_data.iloc[-1]['arrival_time'])
            
            if start_time is not None and end_time is not None:
                travel_time = end_time - start_time
                
                # Reasonable travel time for 510 Spadina (15-45 minutes) I LIVE ON DUNDAS AND SPADINA
                if 15 < travel_time < 45:
                    travel_times.append(travel_time)
    
    print(f"Extracted {len(travel_times)} valid travel times from GTFS")
    
    if len(travel_times) == 0:
        print("WARNING: No valid travel times in GTFS..")

    # This sends the REAL data back to the main function
    return np.array(travel_times)

def calculate_baseline_headways(delay_data):
    """Calculate current headway distribution from delay data"""
    # Sort by date/time
    delay_data = delay_data.sort_values('DateTime').dropna(subset=['DateTime'])
    
    # Group by day and calculate time gaps between consecutive vehicles
    headways = []
    
    for date in delay_data['Date'].unique():
        daily_data = delay_data[delay_data['Date'] == date].sort_values('DateTime')
        daily_data['TimeDiff'] = daily_data['DateTime'].diff().dt.total_seconds() / 60
        
        # Filter REASONABLE ONES (0.5-15 minutes)
        valid_headways = daily_data['TimeDiff'][(daily_data['TimeDiff'] > 0.5) & 
                                                  (daily_data['TimeDiff'] < 15)]
        headways.extend(valid_headways.values)
    
    headways = np.array(headways)
    
    print(f"Extracted {len(headways)} valid headway measurements")
    
    # If few headways, supplement with MinGap data
    if len(headways) < 50 and 'MinGap' in delay_data.columns:
        min_gaps = delay_data['MinGap'].dropna()
        valid_gaps = min_gaps[(min_gaps > 0.5) & (min_gaps < 15)]
        headways = np.concatenate([headways, valid_gaps.values])
        print(f"Added {len(valid_gaps)} headways from MinGap field")
    
    # If still not enough, create synthetic baseline based on known data
    if len(headways) < 50:
        print("WARNING: Insufficient headway data. Using synthetic baseline.")
        # High-frequency route with bunching: mean ~3 min, high CV
        headways = np.random.gamma(shape=2, scale=1.5, size=500)
        headways = np.clip(headways, 0.5, 12)
    
    return headways


In [20]:

# MODEL PROPOSED IMPROVEMENTS
# a lot of it is based on Osuna and Newell (1972) + Ansari Esfeh et al. (2021)  and  Furth and Rahbee (2000)

def model_stop_consolidation_impact(baseline_travel_times, num_stops_removed=5):
    """
    Model impact of stop consolidation
    Assumption: 50 seconds saved per stop removed (decel + dwell + accel)
    """
    time_saved_per_stop = 50 / 60  # Convert to minutes
    total_time_saved = num_stops_removed * time_saved_per_stop
    
    # Reduce travel times and reduce variance (more consistent)
    improved_times = baseline_travel_times - total_time_saved
    improved_times = improved_times * 0.95  # 5% variance reduction
    
    return improved_times

def model_tsp_impact(baseline_travel_times):
    """
    Model impact of Transit Signal Priority
    Assumption: 30-40% reduction in intersection delay
    Estimated 3-5 minutes saved per trip
    """
    # TSP primarily reduces delay variance and intersection wait
    time_saved = np.random.uniform(3, 5, len(baseline_travel_times))
    improved_times = baseline_travel_times - time_saved
    
    # TSP also reduces variance (more predictable)
    improved_times = improved_times * 0.92
    
    return improved_times

def model_headway_control_impact(baseline_headways, target_headway=5.0):
    """
    Model impact of headway-based operations
    Assumption: 40-50% reduction in headway coefficient of variation
    """
    # Current CV (coefficient of variation)
    current_cv = np.std(baseline_headways) / np.mean(baseline_headways)
    
    # Target CV reduction (40-50%)
    improved_cv = current_cv * 0.55  # 45% of original CV
    
    # Generate improved headways with tighter distribution
    improved_headways = np.random.normal(
        loc=target_headway,
        scale=target_headway * improved_cv,
        size=len(baseline_headways)
    )
    
    # Clip to reasonable range
    improved_headways = np.clip(improved_headways, 2.5, 8.0)
    
    return improved_headways

def model_combined_impact(baseline_travel_times, baseline_headways):
    """Model all three interventions combined (multiplicative effect)"""
    # Stop consolidation
    improved_times = model_stop_consolidation_impact(baseline_travel_times)
    
    # Add TSP (on top of stop consolidation)
    improved_times = improved_times - np.random.uniform(3, 5, len(improved_times))
    
    # Further variance reduction from headway control
    improved_times = improved_times * 0.90
    
    # Headway improvements
    improved_headways = model_headway_control_impact(baseline_headways)
    
    return improved_times, improved_headways


In [21]:

# PART 4: VISUALIZATION FUNCTIONS

def plot_travel_time_cdf(baseline_times, improved_times, save_path='figure12_travel_time_cdf.png'):
    """Figure 12: Travel Time Distribution Comparison (CDF)"""
    
    # Validate data
    if len(baseline_times) == 0 or len(improved_times) == 0:
        print("ERROR: No travel time data available for plotting")
        return
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Sort data for CDF
    baseline_sorted = np.sort(baseline_times)
    improved_sorted = np.sort(improved_times)
    
    # Calculate cumulative probabilities
    baseline_cdf = np.arange(1, len(baseline_sorted) + 1) / len(baseline_sorted)
    improved_cdf = np.arange(1, len(improved_sorted) + 1) / len(improved_sorted)
    
    # Plot CDFs
    ax.plot(baseline_sorted, baseline_cdf, linewidth=2.5, color=CURRENT_RED, 
            label='Current Performance', alpha=0.8)
    ax.plot(improved_sorted, improved_cdf, linewidth=2.5, color=PROPOSED_GREEN, 
            label='Proposed (All Interventions)', alpha=0.8)
    
    # Add median lines
    baseline_median = np.median(baseline_times)
    improved_median = np.median(improved_times)
    
    ax.axvline(baseline_median, color=CURRENT_RED, linestyle='--', alpha=0.5, linewidth=1.5)
    ax.axvline(improved_median, color=PROPOSED_GREEN, linestyle='--', alpha=0.5, linewidth=1.5)
    
    # Styling copied and pasted from my other project
    ax.set_xlabel('End-to-End Travel Time (minutes)', fontsize=12, weight='bold')
    ax.set_ylabel('Cumulative Distribution Function', fontsize=12, weight='bold')
    ax.set_title('Travel Time Distribution: Current vs. Proposed System', 
                 fontsize=14, weight='bold', pad=20)
    
    # Remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Legend
    ax.legend(loc='lower right', frameon=True, shadow=False, fontsize=11)
    
    # Grid
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Print statistics
    print(f"\n=== TRAVEL TIME ANALYSIS ===")
    print(f"Current Median: {baseline_median:.1f} min")
    print(f"Proposed Median: {improved_median:.1f} min")
    print(f"Improvement: {baseline_median - improved_median:.1f} min ({(1 - improved_median/baseline_median)*100:.1f}%)")
    print(f"Current 85th percentile: {np.percentile(baseline_times, 85):.1f} min")
    print(f"Proposed 85th percentile: {np.percentile(improved_times, 85):.1f} min")


In [22]:

def plot_headway_cdf(baseline_headways, improved_headways, save_path='figure13_headway_cdf.png'):
    """Figure 13: Headway Regularity Comparison (CDF)"""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Sort data for CDF
    baseline_sorted = np.sort(baseline_headways)
    improved_sorted = np.sort(improved_headways)
    
    # Calculate cumulative probabilities
    baseline_cdf = np.arange(1, len(baseline_sorted) + 1) / len(baseline_sorted)
    improved_cdf = np.arange(1, len(improved_sorted) + 1) / len(improved_sorted)
    
    # Plot CDFs
    ax.plot(baseline_sorted, baseline_cdf, linewidth=2.5, color=CURRENT_RED, 
            label='Current Operations', alpha=0.8)
    ax.plot(improved_sorted, improved_cdf, linewidth=2.5, color=PROPOSED_GREEN, 
            label='Headway-Based Control', alpha=0.8)
    
    # Target headway line
    ax.axvline(5.0, color=DARK_SLATE, linestyle=':', alpha=0.6, linewidth=2,
               label='Target Headway (5 min)')
    
    # Styling
    ax.set_xlabel('Headway Between Vehicles (minutes)', fontsize=12, weight='bold')
    ax.set_ylabel('Cumulative Distribution Function', fontsize=12, weight='bold')
    ax.set_title('Headway Distribution: Schedule-Based vs. Headway-Based Control', 
                 fontsize=14, weight='bold', pad=20)
    
    # Remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Legend
    ax.legend(loc='lower right', frameon=True, shadow=False, fontsize=11)
    
    # Grid
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    
    # Limit x-axis to reasonable range
    ax.set_xlim(0, 12)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Print statistics
    print(f"\n=== HEADWAY ANALYSIS ===")
    print(f"Current Mean Headway: {np.mean(baseline_headways):.2f} min")
    print(f"Current Std Dev: {np.std(baseline_headways):.2f} min")
    print(f"Current CV: {np.std(baseline_headways)/np.mean(baseline_headways):.3f}")
    print(f"\nProposed Mean Headway: {np.mean(improved_headways):.2f} min")
    print(f"Proposed Std Dev: {np.std(improved_headways):.2f} min")
    print(f"Proposed CV: {np.std(improved_headways)/np.mean(improved_headways):.3f}")
    print(f"\nCV Improvement: {(1 - (np.std(improved_headways)/np.mean(improved_headways))/(np.std(baseline_headways)/np.mean(baseline_headways)))*100:.1f}%")

def plot_performance_comparison_table(baseline_times, improved_times, 
                                      baseline_headways, improved_headways,
                                      save_path='figure14_performance_table.png'):
    """Figure 14: Performance Metrics Comparison Table"""
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('tight')
    ax.axis('off')
    
    # Calculate metrics
    metrics = [
        ['Metric', 'Current Baseline', 'Stop Consolidation', 'Stop + TSP', 'Full Integration'],
        ['Median Travel Time (min)', 
         f'{np.median(baseline_times):.1f}',
         f'{np.median(baseline_times) - 4.2:.1f}',
         f'{np.median(baseline_times) - 8.0:.1f}',
         f'{np.median(improved_times):.1f}'],
        ['Average Speed (km/h)', 
         '11.5', '13.5', '15.2', '16.0'],
        ['On-Time Performance', 
         '45%', '55%', '70%', '85%'],
        ['Headway CV', 
         f'{np.std(baseline_headways)/np.mean(baseline_headways):.3f}',
         f'{np.std(baseline_headways)/np.mean(baseline_headways):.3f}',
         f'{np.std(baseline_headways)/np.mean(baseline_headways) * 0.85:.3f}',
         f'{np.std(improved_headways)/np.mean(improved_headways):.3f}'],
        ['Improvement vs. Baseline',
         '0%',
         f'{(1 - (np.median(baseline_times) - 4.2)/np.median(baseline_times))*100:.0f}%',
         f'{(1 - (np.median(baseline_times) - 8.0)/np.median(baseline_times))*100:.0f}%',
         f'{(1 - np.median(improved_times)/np.median(baseline_times))*100:.0f}%']
    ]
    
    table = ax.table(cellText=metrics, cellLoc='center', loc='center',
                     colWidths=[0.25, 0.15, 0.2, 0.15, 0.2])
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2.5)
    
    # Style header row
    for i in range(5):
        cell = table[(0, i)]
        cell.set_facecolor(DARK_SLATE)
        cell.set_text_props(weight='bold', color='white')
    
    # Style baseline column
    for i in range(1, 6):
        cell = table[(i, 1)]
        cell.set_facecolor('#ffe6e6')
    
    # Style final column
    for i in range(1, 6):
        cell = table[(i, 4)]
        cell.set_facecolor('#e6f3ff')
    
    plt.title('Projected System Performance by Intervention Stage', 
              fontsize=14, weight='bold', pad=20)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_delay_by_hour(delay_data, save_path='figure15_delay_by_hour.png'):
    """Figure 15: Delay patterns by time of day"""
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Calculate average delay by hour
    hourly_delay = delay_data.groupby('Hour')['MinDelay'].agg(['mean', 'std']).reset_index()
    
    # Plot current delays
    ax.bar(hourly_delay['Hour'], hourly_delay['mean'], 
           color=CURRENT_RED, alpha=0.7, label='Current Average Delay')
    
    # Model proposed improvement (30% reduction in delays)
    ax.bar(hourly_delay['Hour'], hourly_delay['mean'] * 0.7, 
           color=PROPOSED_GREEN, alpha=0.7, label='Projected with Interventions')
    
    # Styling
    ax.set_xlabel('Hour of Day', fontsize=12, weight='bold')
    ax.set_ylabel('Average Delay (minutes)', fontsize=12, weight='bold')
    ax.set_title('Delay Patterns by Time of Day: Current vs. Proposed', 
                 fontsize=14, weight='bold', pad=20)
    
    # Remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Legend
    ax.legend(loc='upper left', frameon=True, shadow=False, fontsize=11)
    
    # Grid
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5, axis='y')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


In [ ]:
#MAIN EXECUTION

def main():
    """Main execution function"""
    
    # Load delay data
    delay_data = load_delay_data('delay.csv')
    print(f"Loaded {len(delay_data)} delay records for 510 Spadina")
    # I wasn't sure about 510's most delays occuring at 8-11 I was expecting it to be around PM rush hour so this is why I added this diagnostic

    print("\n--Jdelay time diagnostic--")
    hourly_counts = delay_data.groupby('Hour').size()
    print("\nDelay incidents by hour:")
    print(hourly_counts)
    print("\nMean delay by hour:")
    print(delay_data.groupby('Hour')['MinDelay'].mean().round(1))
    print("\nHours with <10 incidents (potentially unreliable):")
    print(hourly_counts[hourly_counts < 10])
    print ("-"*60)
    
    # Load GTFS data
    try:
        stop_times_510, stops, route_id = load_gtfs_data('gtfs/')
        print(f"Loaded GTFS data for route {route_id}")
    except Exception as e:
        print(f"Warning: Could not load GTFS data: {e}")
        print("Will use delay data only for analysis")
        stop_times_510 = None
    
    print("-"*60)
    
    # Calculate baseline metrics
    if stop_times_510 is not None:
        baseline_travel_times = calculate_baseline_travel_times(stop_times_510)
    else:
        # Estimate from delay data
        baseline_travel_times = np.random.normal(30, 4, 500)  # Estimated 30 min avg
    
    baseline_headways = calculate_baseline_headways(delay_data)
    
    print(f"Baseline travel times: {len(baseline_travel_times)} samples")
    print(f"Baseline headways: {len(baseline_headways)} samples")
    
    print("\n........")
    
    # Model improvements
    improved_travel_times, improved_headways = model_combined_impact(
        baseline_travel_times, baseline_headways
    )
    
    print("\n Next, visualizations...")
    
    # Generate all figures    
    plot_travel_time_cdf(baseline_travel_times, improved_travel_times)
    print("[DONE]Figure 12: Travel Time CDF")
    
    plot_headway_cdf(baseline_headways, improved_headways)
    print("[DONE]Figure 13: Headway CDF")
    
    plot_performance_comparison_table(baseline_travel_times, improved_travel_times,
                                     baseline_headways, improved_headways)
    print("[DONE]Figure 14: Performance Comparison Table")
    
    plot_delay_by_hour(delay_data)
    print("[DONE]Figure 15: Delay by Hour")
    

if __name__ == "__main__":
    main()

Loaded 966 delay records for 510 Spadina

--Jdelay time diagnostic--

Delay incidents by hour:
Hour
0     26
1     12
2     25
3      6
4     21
5     40
6     30
7     36
8     44
9     55
10    46
11    53
12    43
13    55
14    68
15    57
16    51
17    55
18    48
19    46
20    40
21    37
22    34
23    38
dtype: int64

Mean delay by hour:
Hour
0     14.5
1      7.9
2     15.8
3     20.7
4      8.2
5     10.9
6      6.6
7      5.3
8      7.3
9      7.3
10    11.9
11     7.2
12     7.5
13    11.0
14    10.4
15     6.8
16     9.3
17     7.7
18     7.1
19     6.6
20    12.5
21    23.7
22    14.3
23     8.8
Name: MinDelay, dtype: float64

Hours with <10 incidents (potentially unreliable):
Hour
3    6
dtype: int64
------------------------------------------------------------
Loaded GTFS data for route 510
------------------------------------------------------------
Processing 1318 unique trips...
Extracted 1267 valid travel times from GTFS
Extracted 65 valid headway measurements
Base